<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/Create_testset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Create testset**

Control全体のリストから、train&valで用いたリストを引き算して、
残りからランダムにcontrolのテストセットを作成する

In [1]:
import torch
from IPython.display import Image, clear_output
import os
import shutil
import glob

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#GDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Setup complete. Using torch 1.12.1+cu113 CPU
Mounted at /content/drive


In [8]:
cont_parent_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Control_photo_1618mai"
cont_train_path = "/content/drive/MyDrive/Deep_learning/666mai_dataset/gravcont_250px/cont"

cont_parent_list = [i for i in os.listdir(cont_parent_path)]
cont_train_list = [i for i in os.listdir(cont_train_path)]

In [45]:
#parentからtrainを引き算
cont_test_list = set(cont_parent_list) - set(cont_train_list)

In [46]:
import random
random.seed(0) 

#ランダムに46画像を選択
cont_test_list = random.sample(cont_test_list, 46)
cont_test_list = sorted(cont_test_list)

#絶対パスに戻す
cont_test_path = [f"{cont_parent_path}/{basename}" for basename in cont_test_list]


###**Image2square**

In [49]:
import time
from PIL import Image 
def convert(in_path, out_path):
    #処理時間の計測
    start = time.time()

    l=0
    for i in in_path:      
          img = Image.open(i)
          img_new = expand2square(img, (0, 0, 0)).resize((250, 250))
          img_new.save(out_path +'/'+ os.path.basename(i))
          print(out_path +'/'+ os.path.basename(i))

    print('Process done!!')
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width-height)//2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, (0, (height - width) // 2))
        return result

In [52]:
# Output
dst_path = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px"

if os.path.exists(dst_path):
    shutil.rmtree(dst_path)
os.makedirs(dst_path)

convert(cont_test_path, dst_path)

/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/1458.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/1795.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/1882.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/2019.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/2172.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/2277.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/2323.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/2353.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/2408.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/2631.jpg
/content/drive/MyDrive/Deep_learning/GO_extended_dataset/Cont_newPatient_250px/3141.JPG
/content/drive/MyDrive/Deep_lear